<a href="https://colab.research.google.com/github/SeanGau/faster-whisper-SRT/blob/main/faster_whisper_SRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faster-whisper opencc

In [ ]:
from faster_whisper import WhisperModel
from google.colab import files
import opencc

keywords = "" # @param {"type":"string"}
language = "zh" # @param {"type":"string","placeholder":"zh"}
model = WhisperModel("medium")
converter = opencc.OpenCC("s2tw")

print("請點擊下方按鈕上傳你的音檔（支援 mp3, wav, m4a 等）：")
uploaded = files.upload()

請點擊下方按鈕上傳你的音檔（支援 mp3, wav, m4a 等）：


Saving g0v_test-stream-1-audio-aac.aac to g0v_test-stream-1-audio-aac.aac


In [ ]:
import os

def transcribe(filename):
  # Transcribe the audio file
  initial_prompt = (
    f"This is a conversation about: {filename.split('/')[-1].split('.')[0]}, {keywords}"
  )

  segments, info = model.transcribe(
    filename,
    language=language,
    initial_prompt=initial_prompt,
    word_timestamps=True,
    vad_filter=True,
    vad_parameters={"min_silence_duration_ms": 500},
  )


  # Function to format time in SRT format with more precise milliseconds
  def format_timestamp(seconds):
    seconds += 300 / 1000
    milliseconds = int(seconds * 1000) % 1000
    seconds = int(seconds)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"


  # Save SRT
  srt_file_path = os.path.splitext(filename)[0] + ".srt"
  with open(srt_file_path, "w", encoding="utf-8") as srt_file:
    id = 1
    for i, segment in enumerate(segments, start=1):
      start = format_timestamp(segment.start)
      current_text = ""
      for word in segment.words:
        current_text += word.word
        end = format_timestamp(word.end)
        # split text by comma or 逗號 into multiple lines
        if "," in word.word or "，" in word.word or word == segment.words[-1]:
          current_text = current_text.replace(",", "").replace("，", "").strip()
          if "zh" in language:
              current_text = converter.convert(current_text)
          line = f"{id}\n{start} --> {end}\n{current_text}"
          print(line, flush=True)
          srt_file.write(f"{line}\n\n")
          start = format_timestamp(word.end)
          current_text = ""
          id += 1

  print(f"SRT file saved to {srt_file_path}")
  files.download(srt_file_path)

# 檢查是否有上傳檔案
if not uploaded:
    print("未選擇任何檔案。")
else:
    for filename in uploaded.keys():
        print(f"\n--- 開始處理檔案：{filename} ---")
        transcribe(filename)
